In [0]:
%sql
CREATE TABLE IF NOT EXISTS employee_table (emp_id INT, salary INT, name string);
INSERT INTO employee_table
values (1,1000,"Sam"),
       (2,2000,"Jimmy"),
       (3,3000,"Neel"),
       (4,4000,"Willy"),
       (5,5000,"Scott");
SELECT * FROM employee_table;

emp_id,salary,name
2,2000,Jimmy
4,4000,Willy
5,5000,Scott
3,3000,Neel
1,1000,Sam


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ${employee_table}_default_location;
CREATE SCHEMA IF NOT EXISTS ${employee_table}_custom_location LOCATION '${da.paths.working_dir}/_custom_location.db';

In [0]:
%sql
DESCRIBE SCHEMA EXTENDED ${employee_table}_default_location;

database_description_item,database_description_value
Namespace Name,_default_location
Comment,
Location,dbfs:/user/hive/warehouse/_default_location.db
Owner,root
Properties,


In [0]:
%sql
DESCRIBE SCHEMA EXTENDED ${employee_table}_custom_location;

database_description_item,database_description_value
Namespace Name,_custom_location
Comment,
Location,dbfs:/_custom_location.db
Owner,root
Properties,


In [0]:
%sql
DESCRIBE DETAIL employee_table;

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,198bfb40-8279-4869-8344-2bd06c65feac,default.employee_table,null,dbfs:/user/hive/warehouse/employee_table,2022-09-06T05:11:04.053+0000,2022-09-06T05:11:18.000+0000,List(),5,5268,Map(),1,2


In [0]:
%sql
DROP TABLE employee_table;

In [0]:
%sql
CREATE OR REPLACE View Employee_view
     AS SELECT emp_id, name
     FROM employee_table;

In [0]:
%sql
select * from Employee_view

emp_id,name
2,Jimmy
4,Willy
5,Scott
3,Neel
1,Sam


In [0]:
%sql
CREATE TEMPORARY VIEW temp_view_salary_2000
AS SELECT * FROM employee_table WHERE salary > 2000 ORDER BY salary ASC;

In [0]:
%sql
SELECT * FROM temp_view_salary_2000

emp_id,salary,name
3,3000,Neel
4,4000,Willy
5,5000,Scott


In [0]:
%sql
CREATE OR REPLACE TEMPORARY View temp_product USING CSV OPTIONS (
  path = '/FileStore/tables/DataSet/transaction.csv',
  header = "true"
);

SELECT * FROM temp_product;

transaction_id,product_id,userid,price,product_description
3300101,1000001,101,700,mouse
3300102,1000002,102,900,keyboard
3300103,1000003,103,34000,tv
3300104,1000004,101,35000,fridge
3300105,1000005,105,55000,sofa
3300106,1000006,106,100,bed
3300107,1000007,105,66000,laptop
3300108,1000008,108,20000,phone
3300109,1000009,101,500,speaker
3300110,1000010,102,1000,chair


In [0]:
%sql
show tables

database,tableName,isTemporary
default,employee_table,false
default,employee_view,false
,temp_product,true
,temp_view_salary_2000,true


In [0]:
%sql
CREATE GLOBAL TEMPORARY VIEW global_temp_view_price_900
AS SELECT * FROM temp_product WHERE price > 900 ORDER BY price ASC;

In [0]:
%sql
select * from global_temp.global_temp_view_price_900

transaction_id,product_id,userid,price,product_description
3300110,1000010,102,1000,chair
3300108,1000008,108,20000,phone
3300103,1000003,103,34000,tv
3300104,1000004,101,35000,fridge
3300105,1000005,105,55000,sofa
3300107,1000007,105,66000,laptop


In [0]:
%sql
show tables in global_temp

database,tableName,isTemporary
global_temp,global_temp_view_price_900,true
,temp_product,true
,temp_view_salary_2000,true


In [0]:
%sql
with product_details(
     total_price,
     Description,
     user_id)
     AS (
     select 
     price,
     product_description,
     userid
     from
     temp_product)
select * from product_details

total_price,Description,user_id
700,mouse,101
900,keyboard,102
34000,tv,103
35000,fridge,101
55000,sofa,105
100,bed,106
66000,laptop,105
20000,phone,108
500,speaker,101
1000,chair,102


In [0]:
%sql
with product_details(
     total_price,
     Description,
     user_id)
     AS (
     select 
     price,
     product_description,
     userid
     from
     temp_product)
select * from product_details
where total_price > 2000

total_price,Description,user_id
34000,tv,103
35000,fridge,101
55000,sofa,105
66000,laptop,105
20000,phone,108


In [0]:
%sql
Select
     min(total_price) As Minimum_Price
     from(
          with min_price(total_price) As(
          Select 
                price
          from
              temp_product)
          select * from min_price);

Minimum_Price
100
